In [99]:
import pandas as pd
import numpy as np
import tensorflow as tf
import keras
import PIL
import cv2
import matplotlib.pyplot as plt
import seaborn as sns
import os
import random
from tqdm import tqdm
import tensorflow_addons as tfa
import random
from sklearn.preprocessing import MultiLabelBinarizer
import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import urllib
import urllib.request as ur
import sys

In [100]:
cred = credentials.Certificate('/home/vh/Documents/serviceAccount.json')

# app = firebase_admin.initialize_app(cred)

db = firestore.client()

images_ref = db.collection(u'images')

query = images_ref.where(u'status', u'==', u'')

docs = query.stream()

urls = []

for doc in docs:
    doc_dict = doc.to_dict()

    imageUrl = doc_dict['imageUrl']
    
    urls.append(imageUrl)

print(urls)

/home/vh/.local/lib/python3.11/site-packages/google/cloud/firestore_v1/base_collection.py:290: UserWarning: Detected filter using positional arguments. Prefer using the 'filter' keyword argument instead.
  return query.where(field_path, op_string, value)


['https://firebasestorage.googleapis.com/v0/b/plant-pests-detection.appspot.com/o/images%2Faedb1485640bcf78.jpgc8b8061b-420e-4c22-83ea-c23579cecb37?alt=media&token=c766603b-a2b9-429d-870a-cfd56a7fb89c', 'https://firebasestorage.googleapis.com/v0/b/plant-pests-detection.appspot.com/o/images%2Ff5bf9213cb900e62.jpg9d07a440-ef73-434b-b0db-feb47bae66e3?alt=media&token=5e803040-81d7-4e6c-ab49-fb444e0a1e11']


In [101]:
model = tf.keras.models.load_model('model/saved_model', compile=False)

In [102]:
images = []
for i in range(0,len(urls)):
    image = "image_" + str(i) +".jpg"
    images.append(image)
    urllib.request.urlretrieve(urls[i], image)
    with PIL.Image.open(image) as img:
        img = img.resize((256,256))
        img.save(f'./{predict_image}/{image}')

In [103]:
print(images)
test = pd.DataFrame({'image':images})
print(test)

['image_0.jpg', 'image_1.jpg']
         image
0  image_0.jpg
1  image_1.jpg


In [104]:
datagen = keras.preprocessing.image.ImageDataGenerator(rescale=1/255.0,
                                                        preprocessing_function=None,
                                                        data_format=None,
                                                    )
test_data = datagen.flow_from_dataframe(
    test,
    directory = '.',
    x_col="image",
    y_col= None,
    color_mode="rgb",
    target_size = (256,256),
    classes=None,
    class_mode=None,
    batch_size=32,
    shuffle=False,
    seed=40,
)

print(test_data)

preds = model.predict(test_data)
print(preds)
preds = preds.tolist()

indices = []
for pred in preds:
    temp = []
    for category in pred:
        if category>=0.3:
            temp.append(pred.index(category))
    if temp!=[]:
        indices.append(temp)
    else:
        temp.append(np.argmax(pred))
        indices.append(temp)
    
print(indices)

Found 2 validated image filenames.


2023-05-10 18:23:03.519937: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


1/1 [==============================] - 2s 2s/step
[[7.3246844e-02 2.2475398e-01 2.3127902e-02 2.6427642e-02 8.6483115e-04
  9.6409649e-01]
 [1.0849129e-02 4.8134673e-02 8.4769621e-02 3.4275169e-03 1.1059185e-03
  8.7391227e-01]]
[[5], [5]]


In [105]:
labels = {'complex': 0, 'frog_eye_leaf_spot': 1, 'healthy': 2, 'powdery_mildew': 3, 'rust': 4, 'scab': 5}
labels = dict((v,k) for k,v in labels.items())
print(labels)

testlabels = []


for image in indices:
    temp = []
    for i in image:
        temp.append(str(labels[i]))
    testlabels.append(' '.join(temp))

print(testlabels)

{0: 'complex', 1: 'frog_eye_leaf_spot', 2: 'healthy', 3: 'powdery_mildew', 4: 'rust', 5: 'scab'}
['scab', 'scab']
